In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from keras.utils import to_categorical
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import PIL.Image
import skimage
import tensorflow as tf
import keras.backend as K
from keras.utils import to_categorical
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

c:\users\shantanu\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#data preprocess
def preprocess(data):
    img = image.load_img(data, target_size=(227,227,2))
    x = image.img_to_array(img)
    return x
    

In [ ]:
#dataload
df=pd.read_excel('train_image_paths.xlsx',sheetname="train_image_paths")
X_train=[]
Y_train=[]
X_test=[]
Y_test=[]
feature=df['X'].tolist()
label=df['Y'].tolist()
for i in range(0,10000):
    X_train.append(preprocess(feature[i]))
    Y_train.append(label[i])
for i in range(10000,10050):
    X_test.append(preprocess(feature[i]))
    Y_test.append((label[i]))
X_train=np.array(X_train)
Y_train=np.array(Y_train)
X_test=np.array(X_test)
Y_test=np.array(Y_test)


    

In [8]:
def mModel(input_shape):#Alex net architecture 
    X_input = Input(input_shape)
    # CONv1
    X = ZeroPadding2D((5, 5))(X_input)
    X = Conv2D(9, (11, 11), strides = (4, 4), name = 'conv0')(X)
    X = BatchNormalization(axis = 1, name = 'bn0')(X)
    X = Activation('relu')(X)
    # MAXPOOL1
    X = MaxPooling2D((2, 2),strides=(2,2),name='max_pool')(X)
    #Conv layer 2
    X = Conv2D(25, (5, 5), strides = (2, 2), name = 'conv1')(X)
    X = BatchNormalization(axis = 1, name = 'bn1')(X)
    X = Activation('relu')(X)
    #MAX pool2
    X = MaxPooling2D((2, 2),strides=(2,2),name='max_pool1')(X)
    # FLATTEN=FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)
    model = Model(inputs = X_input, outputs = X, name='mModel')
    return model

In [9]:
MModel = mModel(X_train.shape[1:])

In [10]:
MModel.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [11]:

MModel.fit(x=X_train,y=Y_train,epochs=5,batch_size=50)

Epoch 1/5
10000/10000 [==============================] - 264s 26ms/step - loss: 0.6510 - acc: 0.6007
Epoch 2/5
10000/10000 [==============================] - 272s 27ms/step - loss: 0.6385 - acc: 0.6112
Epoch 3/5
10000/10000 [==============================] - 285s 28ms/step - loss: 0.6319 - acc: 0.6259
Epoch 4/5
10000/10000 [==============================] - 298s 30ms/step - loss: 0.6295 - acc: 0.6312
Epoch 5/5
10000/10000 [==============================] - 249s 25ms/step - loss: 0.6223 - acc: 0.6447


In [ ]:
preds = MModel.evaluate(x=X_test,y=Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
to_categorical(MModel.predict(X_test))

In [14]:
MModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 237, 237, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 57, 57, 9)         3276      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 57, 57, 9)         228       
_________________________________________________________________
activation_3 (Activation)    (None, 57, 57, 9)         0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 28, 28, 9)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 12, 12, 25)        5650      
__________